<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/multilable_test_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install torch
!pip install transformers
!pip install peft
!pip install huggingface_hub

In [ ]:
from google.colab import userdata
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType  # PEFT 라이브러리 필요
import pandas as pd
from google.colab import drive
from sklearn.metrics import f1_score

In [ ]:
from peft import PeftModel, LoraConfig, get_peft_model
drive.mount('/content/drive', force_remount=False)

token =userdata.get('Hugging_Face_llama3.2')
model_path = 'meta-llama/Llama-3.2-1B'

num_labels = 9 # len(label_names)
base_model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=num_labels, use_auth_token=token)


tokenizer = AutoTokenizer.from_pretrained(model_path, use_auth_token=token)
tokenizer.pad_token = tokenizer.eos_token
base_model.config.pad_token_id = tokenizer.pad_token_id

#lora_path = "/content/drive/MyDrive/multi_label/checkpoint_12500"  # Adjust this path as needed

#model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/DataSet_new/lora_model")
#model = PeftModel.from_pretrained(base_model, lora_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
import pandas as pd

#drive.mount('/content/drive', force_remount=False)
test_path = ('/content/drive/MyDrive/DataSet_new/rev_tag_test_samples.csv')

df_test = pd.read_csv(test_path)
label_names = list(df_test.columns[5:])

#df_test
df_test['text'] = df_test['Title'] + ' ' + df_test['Body']
text_test = list(df_test['text'])
label_test = list(df_test[label_names].values)

In [ ]:
import torch

# ... (your existing code) ...

# 모델을 평가 모드로 설정
model = base_model

model.eval()
model.to('cuda')  # 모델을 명시적으로 GPU로 이동
#base_model.eval()
#base_model.to('cuda')  # 모델을 명시적으로 GPU로 이동

test_encodings = tokenizer(text_test, truncation=True, padding=True, max_length=512, return_tensors="pt")

# 예측 수행
predictions = []
batch_size = 16 # adjust this based on your available GPU memory
num_batches = len(test_encodings['input_ids']) // batch_size + (len(test_encodings['input_ids']) % batch_size != 0)

with torch.no_grad():
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(test_encodings['input_ids']))

        batch_encodings = {k: v[start_idx:end_idx].to('cuda') for k, v in test_encodings.items()}  # Move each batch to GPU

        outputs = model(**batch_encodings)
        #outputs = base_model(**batch_encodings)

        logits = outputs.logits
        probs = torch.sigmoid(logits)  # 시그모이드 함수를 사용해 확률로 변환
        preds = (probs >= 0.5).int()  # 임계값 0.5 이상일 때 라벨로 간주
        predictions.extend(preds.cpu().numpy())  # 결과를 CPU로 이동 and extend predictions list

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support

def compute_metrics_test(preds,labels):
    #labels = pred.label_ids
    #preds = pred.predictions >= 0.5  # 시그모이드 출력 후 임계값 0.5 이상일 때 예측으로 분류
    preds = np.array(preds)
    labels = np.array(labels) # Convert labels to a NumPy array

    accuracy = accuracy_score(labels,preds)
    precision, recall, f1_score_result, _ = precision_recall_fscore_support(labels,preds, average='micro')
    precision_macro, recall_macro, f1_score_result_macro, _ = precision_recall_fscore_support(labels,preds, average='macro')
    precision_weighted, recall_weighted, f1_score_result_weighted, _ = precision_recall_fscore_support(labels,preds, average='weighted')

    flat_predictions = preds.reshape(-1)
    flat_labels = labels.reshape(-1)
    flat_accuracy = accuracy_score(flat_labels, flat_predictions)


    return {
        'flat_accuracy': flat_accuracy,
        'accuracy': accuracy,
        'micro_precision': precision,
        'micro_recall': recall,
        'micro_f1': f1_score_result,
        'macro_precision': precision_macro,
        'macro_recall': recall_macro,
        'macro_f1': f1_score_result_macro,
        'weighted_precision': precision_weighted,
        'weighted_recall': recall_weighted,
        'weighted_f1': f1_score_result_weighted,
        }

In [ ]:
#test_result =compute_metrics_test(predictions, label_test)
test_result_2 =compute_metrics_test(predictions, label_test)

In [ ]:
##Baseline result
print(test_result_2)

{'flat_accuracy': 0.4247037037037037, 'accuracy': 0.0, 'micro_precision': 0.13760908036858485, 'micro_recall': 0.6167943107221007, 'micro_f1': 0.22501621513745448, 'macro_precision': 0.1468339701616346, 'macro_recall': 0.630494701275153, 'macro_f1': 0.2037179778706396, 'weighted_precision': 0.216496304853086, 'weighted_recall': 0.6167943107221007, 'weighted_f1': 0.2929860463494297}


### Trained result

In [ ]:
print(test_result)

{'flat_accuracy': 0.9618518518518518, 'accuracy': 0.7103333333333334, 'micro_precision': 0.838053553038105, 'micro_recall': 0.8903172866520788, 'micro_f1': 0.863395225464191, 'macro_precision': 0.7322616668565142, 'macro_recall': 0.892252327889055, 'macro_f1': 0.7855672771899964, 'weighted_precision': 0.8489568014391514, 'weighted_recall': 0.8903172866520788, 'weighted_f1': 0.8670596838207321}
